# Structural Identifiability Analysis: A Comprehensive Tutorial with Julia

**Author:** Abdallah Alsammani  
**Affiliation:** Department of Mathematical Sciences, Delaware State University, Dover, DE, USA  
**Contact:** aalsammani@desu.edu

---

## About This Notebook

This notebook provides a self-contained, hands-on tutorial on **structural identifiability analysis** for ordinary differential equation (ODE) models. It is designed for graduate students and researchers who work with mathematical models in biology, pharmacology, epidemiology, or related fields, and who may be encountering identifiability analysis for the first time.

**What you will learn:**
1. What structural identifiability means and why it matters for parameter estimation
2. The mathematical theory behind identifiability (global, local, and nonidentifiable)
3. How to use the Julia package `StructuralIdentifiability.jl` to analyze your own models
4. Detailed walkthroughs of six progressively complex models drawn from real applications
5. How to interpret results and what to do when parameters are nonidentifiable

**Prerequisites:**
- Basic familiarity with ordinary differential equations
- Basic understanding of parameter estimation (curve fitting)
- No prior knowledge of identifiability theory is assumed

**Software requirements:**
- Julia 1.6 or later
- `StructuralIdentifiability.jl` package

## Table of Contents

1. [Motivation: Why Identifiability Matters](#1-motivation)
2. [Mathematical Background](#2-mathematical-background)
3. [Installation and Setup](#3-installation-and-setup)
4. [Example 1: First-Order Decay (The Simplest Case)](#4-example-1)
5. [Example 2: A Nonidentifiable Variant (When Things Go Wrong)](#5-example-2)
6. [Example 3: SIR Epidemic Model](#6-example-3)
7. [Example 4: Two-Compartment Pharmacokinetic Model](#7-example-4)
8. [Example 5: Within-Host Viral Dynamics](#8-example-5)
9. [Example 6: SEIR Model with Hospitalization](#9-example-6)
10. [Example 7: Environmental Transmission Model](#10-example-7)
11. [Interpreting and Resolving Nonidentifiability](#11-resolving)
12. [Summary and Best Practices](#12-summary)
13. [References](#13-references)

---
## 1. Motivation: Why Identifiability Matters <a id="1-motivation"></a>

### The Hidden Trap in Parameter Estimation

Imagine you have built a mathematical model of how a drug moves through the human body. The model has several parameters: absorption rates, elimination rates, volumes of distribution. You collect data from a clinical trial, run your favorite optimization algorithm, and obtain parameter estimates. The algorithm converges, the fit looks great, and you report the results in a paper.

**But here is the problem:** What if the same data could be explained equally well by completely different parameter values? What if there are infinitely many parameter combinations that produce the exact same model output?

If this is the case, your parameter estimates are **arbitrary**. A different starting point for the optimizer would have given you different numbers, and both would fit the data equally well. Any scientific conclusions drawn from those specific numbers would be unreliable.

### A Concrete Example

Consider measuring the concentration of a substance that decays over time. If we observe:

$$y(t) = A \cdot e^{-kt}$$

and we know $A$ and can measure $y(t)$, then $k$ is uniquely determined: there is exactly one value of $k$ that produces the observed curve. The parameter $k$ is **identifiable**.

But what if the observation is:

$$y(t) = a \cdot x_0 \cdot e^{-kt}$$

where both $a$ (a scaling factor) and $x_0$ (initial condition) are unknown? Then we can only determine the product $a \cdot x_0$, not $a$ and $x_0$ individually. If someone reports $a = 2$ and $x_0 = 5$, another researcher could equally claim $a = 1$ and $x_0 = 10$. Both are consistent with the data. The individual parameters $a$ and $x_0$ are **nonidentifiable**.

### Structural vs. Practical Identifiability

There are two distinct questions:

| | **Structural Identifiability** | **Practical Identifiability** |
|---|---|---|
| **Question** | Can parameters *theoretically* be determined from the model structure and observations? | Can parameters be estimated *reliably* from the actual data at hand? |
| **Depends on** | Model equations + what is measured | Data quality, noise, sample size |
| **Data assumed** | Perfect, continuous, noise-free | Real: finite, discrete, noisy |
| **Method** | Algebraic/analytical | Statistical/numerical |

**Key insight:** If a parameter is structurally nonidentifiable, it is *guaranteed* to be practically nonidentifiable too. No amount of data can fix a structural problem. This is why structural analysis should come first.

This notebook focuses on **structural identifiability**: determining whether your model structure permits unique parameter recovery, before you ever look at data.

---
## 2. Mathematical Background <a id="2-mathematical-background"></a>

### 2.1 The General ODE Model

We consider models of the form:

$$\dot{\mathbf{x}}(t) = \mathbf{f}(\mathbf{x}(t), \mathbf{u}(t), \boldsymbol{\theta})$$
$$\mathbf{y}(t) = \mathbf{g}(\mathbf{x}(t), \boldsymbol{\theta})$$
$$\mathbf{x}(0) = \mathbf{x}_0(\boldsymbol{\theta})$$

where:
- $\mathbf{x}(t) \in \mathbb{R}^n$ is the **state vector** (the variables that evolve over time)
- $\mathbf{u}(t) \in \mathbb{R}^m$ is the **input** (known external forcing, like a drug dose)
- $\boldsymbol{\theta} \in \mathbb{R}^p$ is the **parameter vector** (the unknowns we want to estimate)
- $\mathbf{y}(t) \in \mathbb{R}^q$ is the **output** (what we can actually measure)

The functions $\mathbf{f}$ and $\mathbf{g}$ define the model dynamics and observations, respectively.

> **Important:** Identifiability depends on *both* the dynamics ($\mathbf{f}$) *and* the observations ($\mathbf{g}$). The same dynamical system can be identifiable or nonidentifiable depending on what you measure!

### 2.2 Three Levels of Identifiability

Given a model, each parameter $\theta_i$ falls into exactly one of three categories:

#### Globally Identifiable (GI)
A parameter $\theta_i$ is **globally identifiable** if, for almost all true parameter values $\boldsymbol{\theta}^*$:

$$\mathbf{y}(t; \boldsymbol{\theta}) = \mathbf{y}(t; \boldsymbol{\theta}^*) \text{ for all } t \geq 0 \quad \Longrightarrow \quad \theta_i = \theta_i^*$$

*In plain language:* If two parameter vectors produce identical outputs, then $\theta_i$ must have the same value in both. There is exactly **one** value of $\theta_i$ consistent with the data. This is the best case.

#### Locally Identifiable (LI)
A parameter $\theta_i$ is **locally identifiable** if there exists a neighborhood around $\boldsymbol{\theta}^*$ in which it is the unique solution. However, there may be **finitely many** other isolated values of $\theta_i$ (outside this neighborhood) that also produce the same output.

*In plain language:* The parameter can take a small **finite** number of discrete values consistent with the data. For example, both $\theta_i = 3$ and $\theta_i = 7$ might work, but nothing in between.

#### Nonidentifiable (NI)
A parameter $\theta_i$ is **nonidentifiable** if infinitely many values of $\theta_i$, forming a continuous set, produce the same output.

*In plain language:* The parameter can vary over an entire range (or manifold) without changing the output at all. Estimation is fundamentally impossible.

### 2.3 Identifiable Functions

When individual parameters are nonidentifiable, certain **combinations** of parameters may still be recoverable. These are called **identifiable functions**.

**Example:** If $a$ and $x_0$ are nonidentifiable but $a \cdot x_0$ is identifiable, then:
- You **cannot** estimate $a$ or $x_0$ separately
- You **can** estimate the product $a \cdot x_0$
- The product $a \cdot x_0$ is an identifiable function

Finding identifiable functions tells you exactly what information the data *can* provide, even when the full parameter vector cannot be recovered.

### 2.4 The Differential Algebra Approach (How the Software Works)

The software we will use (`StructuralIdentifiability.jl`) employs the **differential algebra** method. Here is the intuition:

1. **Eliminate hidden states:** Using algebraic manipulation, remove all state variables that are not directly observed, obtaining equations that involve only the input $\mathbf{u}$, the output $\mathbf{y}$, their time derivatives, and the parameters $\boldsymbol{\theta}$.

2. **Extract the coefficient map:** These "input-output equations" have the form:
$$\sum_i c_i(\boldsymbol{\theta}) \cdot m_i(y, \dot{y}, \ddot{y}, \ldots, u, \dot{u}, \ldots) = 0$$
where the coefficients $c_i(\boldsymbol{\theta})$ depend only on the parameters.

3. **Test injectivity:** Since the $c_i$ are observable from data, identifiability reduces to asking: does the map $\boldsymbol{\theta} \mapsto (c_1(\boldsymbol{\theta}), \ldots, c_r(\boldsymbol{\theta}))$ uniquely determine $\boldsymbol{\theta}$?

The software automates all of these steps.

---
## 3. Installation and Setup <a id="3-installation-and-setup"></a>

First, we need to install the `StructuralIdentifiability.jl` package. Run the cell below once. After the first installation, you can skip it in future sessions.

In [ ]:
# Install the package (run once, then comment out)
using Pkg
Pkg.add("StructuralIdentifiability")

Now load the package. This may take a moment on first use as Julia compiles the code.

In [ ]:
using StructuralIdentifiability

println("StructuralIdentifiability.jl loaded successfully!")
println("Julia version: ", VERSION)

### How the Software Works: Three Key Functions

Throughout this notebook, we will use three main functions:

| Function | What It Does | Speed |
|----------|-------------|-------|
| `assess_local_identifiability(model)` | Tests whether each parameter is locally identifiable | Fast |
| `assess_identifiability(model)` | Tests whether each parameter is globally, locally, or nonidentifiable | Slower but more informative |
| `find_identifiable_functions(model)` | Finds the combinations of parameters that *are* identifiable | Most informative for nonidentifiable models |

### How to Define a Model: The `@ODEmodel` Macro

Models are defined using the `@ODEmodel` macro with the following syntax:

```julia
model = @ODEmodel(
    x1'(t) = <right-hand side>,    # State equations (use prime for derivative)
    x2'(t) = <right-hand side>,
    y1(t) = <expression>,           # Output equations (what you observe)
    y2(t) = <expression>
)
```

**Rules:**
- State variables use the prime notation: `x'(t)` means $dx/dt$
- Parameters appear as symbols (e.g., `k`, `beta`, `gamma`)
- Known inputs are included as function arguments (e.g., `u(t)`)
- Output equations define what you can measure
- All expressions must be rational (polynomials or ratios of polynomials)

---
## 4. Example 1: First-Order Decay (The Simplest Case) <a id="4-example-1"></a>

### Model Description

A substance decays at a rate proportional to its current amount:

$$\frac{dx}{dt} = -k \, x, \quad x(0) = x_0$$

where:
- $x(t)$ = concentration at time $t$
- $k > 0$ = elimination rate constant
- $x_0 > 0$ = initial concentration

**What we measure:** The concentration directly: $y(t) = x(t)$.

**What we want to know:** Can we uniquely determine $k$ and $x_0$ from the measurements?

### Why We Expect Identifiability

The analytical solution is $x(t) = x_0 \, e^{-kt}$, so $y(t) = x_0 \, e^{-kt}$.

From the measurements:
- $y(0) = x_0$ gives us $x_0$ directly
- $\dot{y}(0) / y(0) = -k$ gives us $k$

Both parameters are determined. Let us verify this computationally.

### Computational Analysis

In [ ]:
# ═══════════════════════════════════════════════════════════════
# Example 1: Simple first-order decay with direct observation
# ═══════════════════════════════════════════════════════════════

# Define the model
simple_decay = @ODEmodel(
    x'(t) = -k * x(t),      # dx/dt = -k*x
    y(t) = x(t)              # We observe x directly
)

println("Model defined: dx/dt = -k*x, y = x")
println("Unknown parameters: k, x(0)")
println()

# Step 1: Quick local identifiability check
println("─" ^ 50)
println("STEP 1: Local Identifiability")
println("─" ^ 50)
local_result = assess_local_identifiability(simple_decay)
println(local_result)
println()

# Step 2: Full global identifiability analysis
println("─" ^ 50)
println("STEP 2: Global Identifiability")
println("─" ^ 50)
global_result = assess_identifiability(simple_decay)
println(global_result)

### Interpreting the Results

The output should show:
- **`k`** → `:globally` — The elimination rate is globally identifiable. There is exactly one value of $k$ consistent with any output trajectory.
- **`x(0)`** → `:globally` — The initial condition is globally identifiable.

**What this means in practice:** If you have concentration-time data from a first-order decay process, you can, in principle, uniquely determine both the rate constant and the initial concentration. This is the ideal situation for parameter estimation.

> **Key takeaway:** Direct observation of the state variable, combined with a simple model structure, yields full global identifiability. This is our baseline "everything works" case.

---
## 5. Example 2: A Nonidentifiable Variant <a id="5-example-2"></a>

### What Happens When We Add an Unknown Scaling Factor?

Now consider the same decay dynamics, but suppose we observe a *scaled* version of the concentration:

$$\frac{dx}{dt} = -k \, x, \quad y(t) = a \cdot x(t)$$

where $a > 0$ is an unknown scaling factor (perhaps a detector calibration constant).

**Unknown parameters:** $k$, $a$, and $x(0) = x_0$.

### Why We Expect a Problem

The output is $y(t) = a \cdot x_0 \cdot e^{-kt}$. Suppose the true parameters are $k^* = 1$, $a^* = 2$, $x_0^* = 5$. Then $y(t) = 10 \, e^{-t}$.

But this is also produced by $k = 1$, $a = 1$, $x_0 = 10$, or $k = 1$, $a = 10$, $x_0 = 1$, or infinitely many other combinations where $a \cdot x_0 = 10$.

We can determine $k$ (from the exponential rate) and the product $a \cdot x_0$ (from the amplitude), but **not** $a$ and $x_0$ individually.

### Computational Analysis

In [ ]:
# ═══════════════════════════════════════════════════════════════
# Example 2: Decay with unknown scaling (nonidentifiable)
# ═══════════════════════════════════════════════════════════════

scaled_decay = @ODEmodel(
    x'(t) = -k * x(t),      # Same dynamics
    y(t) = a * x(t)          # But observe a*x, not x
)

println("Model defined: dx/dt = -k*x, y = a*x")
println("Unknown parameters: k, a, x(0)")
println()

# Global identifiability
println("─" ^ 50)
println("Global Identifiability Analysis")
println("─" ^ 50)
global_result = assess_identifiability(scaled_decay)
println(global_result)
println()

# Find identifiable functions
println("─" ^ 50)
println("Identifiable Functions")
println("─" ^ 50)
println("(These are the parameter combinations that CAN be determined)")
id_funcs = find_identifiable_functions(scaled_decay)
println(id_funcs)

### Interpreting the Results

The output should show:
- **`k`** → `:globally` — The rate constant is globally identifiable (it determines the shape of the exponential)
- **`a`** → `:nonidentifiable` — The scaling factor cannot be determined individually
- **`x(0)`** → `:nonidentifiable` — The initial condition cannot be determined individually

The identifiable functions reveal that while $a$ and $x_0$ are individually nonidentifiable, the product $a \cdot x(0)$ is identifiable.

### Lessons Learned

This example illustrates a fundamental source of nonidentifiability: **unobservable scaling**. When the observation equation includes an unknown multiplicative factor, the factor and the initial condition become confounded. This pattern appears frequently in practice:

- In pharmacokinetics: the volume of distribution $V$ and the dose amount $D$ may be confounded if both are unknown
- In fluorescence microscopy: the detector gain and the true fluorescence intensity cannot be separated
- In epidemic models: reporting rates and true case counts are confounded

> **Key takeaway:** Always check whether your observation equation introduces unknown scaling factors. If it does, the individual components of the scale will be nonidentifiable, though their product may be identifiable.

---
## 6. Example 3: SIR Epidemic Model <a id="6-example-3"></a>

### Model Description

The Susceptible-Infectious-Recovered (SIR) model is one of the most widely used models in mathematical epidemiology:

$$\frac{dS}{dt} = -\beta \, S \, I$$
$$\frac{dI}{dt} = \beta \, S \, I - \gamma \, I$$
$$\frac{dR}{dt} = \gamma \, I$$

where:
- $S(t)$ = susceptible population
- $I(t)$ = infectious population
- $R(t)$ = recovered population
- $\beta > 0$ = transmission rate
- $\gamma > 0$ = recovery rate

The **basic reproduction number** is $\mathcal{R}_0 = \beta \, S(0) / \gamma$.

### The Critical Question: What Do We Observe?

In epidemiology, different surveillance systems provide different types of data. We will analyze three scenarios to show how the choice of observation fundamentally changes identifiability.

### Scenario A: Observing Only Infectious Cases

In [ ]:
# ═══════════════════════════════════════════════════════════════
# Example 3A: SIR model — observing I(t) only
# ═══════════════════════════════════════════════════════════════

sir_I_only = @ODEmodel(
    S'(t) = -beta * S(t) * I(t),
    I'(t) = beta * S(t) * I(t) - gamma * I(t),
    R'(t) = gamma * I(t),
    y(t) = I(t)                     # Only observe infectious count
)

println("═" ^ 55)
println("  SIR Model — Scenario A: Observing I(t) only")
println("═" ^ 55)
println()

println("Global Identifiability:")
result_A = assess_identifiability(sir_I_only)
println(result_A)
println()

println("Identifiable Functions:")
id_funcs_A = find_identifiable_functions(sir_I_only)
println(id_funcs_A)

### Scenario A: Interpretation

With only $I(t)$ observed:
- **$\gamma$** is globally identifiable — the recovery rate determines how quickly $I(t)$ declines after the peak
- **$\beta$** is nonidentifiable individually — it appears confounded with $S(0)$
- **$S(0)$** is nonidentifiable individually
- **$\beta \cdot S(0)$** is an identifiable function — this product determines the initial force of infection
- **$I(0)$** is globally identifiable — it equals $y(0)$
- **$R(0)$** is nonidentifiable — it does not affect $S$ or $I$ dynamics and is not observed

**Physical interpretation:** From the infectious curve alone, we can determine *how fast* people recover ($\gamma$) and the *total initial force of infection* ($\beta \cdot S(0)$), but we cannot separate the transmission rate from the initial susceptible population size.

### Scenario B: Observing Both Infectious and Recovered

In [ ]:
# ═══════════════════════════════════════════════════════════════
# Example 3B: SIR model — observing I(t) and R(t)
# ═══════════════════════════════════════════════════════════════

sir_I_R = @ODEmodel(
    S'(t) = -beta * S(t) * I(t),
    I'(t) = beta * S(t) * I(t) - gamma * I(t),
    R'(t) = gamma * I(t),
    y1(t) = I(t),                   # Observe infectious
    y2(t) = R(t)                    # AND recovered
)

println("═" ^ 55)
println("  SIR Model — Scenario B: Observing I(t) and R(t)")
println("═" ^ 55)
println()

println("Global Identifiability:")
result_B = assess_identifiability(sir_I_R)
println(result_B)

### Scenario B: Interpretation

With both $I(t)$ and $R(t)$ observed, we have more information. Since $S(t) = N - I(t) - R(t)$ (where $N$ is the total population, assumed known), observing $I$ and $R$ effectively tells us $S(t)$ as well. This additional information can resolve the $\beta$ vs. $S(0)$ ambiguity.

Check the output: if $N$ is known, all parameters should be globally identifiable.

### Scenario C: Observing Incidence (New Cases per Unit Time)

In [ ]:
# ═══════════════════════════════════════════════════════════════
# Example 3C: SIR model — observing incidence (new infections)
# ═══════════════════════════════════════════════════════════════

sir_incidence = @ODEmodel(
    S'(t) = -beta * S(t) * I(t),
    I'(t) = beta * S(t) * I(t) - gamma * I(t),
    R'(t) = gamma * I(t),
    y(t) = beta * S(t) * I(t)      # Observe rate of new infections
)

println("═" ^ 55)
println("  SIR Model — Scenario C: Observing Incidence β·S·I")
println("═" ^ 55)
println()

println("Global Identifiability:")
result_C = assess_identifiability(sir_incidence)
println(result_C)
println()

println("Identifiable Functions:")
id_funcs_C = find_identifiable_functions(sir_incidence)
println(id_funcs_C)

### Summary of SIR Identifiability Across Observation Scenarios

| Parameter | Observe $I$ only | Observe $I$ and $R$ | Observe incidence $\beta SI$ |
|-----------|:-----------------:|:-------------------:|:----------------------------:|
| $\beta$ | Nonidentifiable | Globally identifiable | Depends on setup |
| $\gamma$ | Globally identifiable | Globally identifiable | Globally identifiable |
| $S(0)$ | Nonidentifiable | Globally identifiable | Depends on setup |
| $I(0)$ | Globally identifiable | Globally identifiable | Identifiable |
| $R(0)$ | Nonidentifiable | Globally identifiable | Nonidentifiable |
| $\beta \cdot S(0)$ | Identifiable | Identifiable | Identifiable |

> **Key takeaway:** The same dynamical system can be identifiable or nonidentifiable depending on *what you measure*. Before fitting any model, you must specify your observation function and verify identifiability. This is not an abstract exercise — it has direct consequences for whether your parameter estimates are meaningful.

---
## 7. Example 4: Two-Compartment Pharmacokinetic Model <a id="7-example-4"></a>

### Model Description

In pharmacokinetics, drugs distribute between a **central compartment** (blood, well-perfused organs) and a **peripheral compartment** (muscle, fat, poorly perfused tissues). The two-compartment model is:

$$\frac{dA_1}{dt} = -(k_{12} + k_e) \, A_1 + k_{21} \, A_2 + u(t)$$
$$\frac{dA_2}{dt} = k_{12} \, A_1 - k_{21} \, A_2$$

where:
- $A_1(t)$ = drug amount in the central compartment
- $A_2(t)$ = drug amount in the peripheral compartment
- $k_{12}$ = transfer rate from central to peripheral
- $k_{21}$ = transfer rate from peripheral to central
- $k_e$ = elimination rate from the central compartment
- $u(t)$ = drug input (e.g., intravenous infusion)
- $V$ = volume of distribution of the central compartment

**Observation:** Plasma concentration: $y(t) = A_1(t) / V$

**Note:** This model includes an **input** $u(t)$, which is a known function (the dosing regimen).

### Why This Model Is Important

This is the workhorse of clinical pharmacokinetics. Drug dosing regimens, therapeutic monitoring, and drug-drug interaction studies all depend on reliable estimates of these parameters. If they are nonidentifiable, clinical decisions based on the estimates could be wrong.

### Computational Analysis

In [ ]:
# ═══════════════════════════════════════════════════════════════
# Example 4: Two-compartment pharmacokinetic model
# ═══════════════════════════════════════════════════════════════

pk_2comp = @ODEmodel(
    A1'(t) = -(k12 + ke) * A1(t) + k21 * A2(t) + u(t),
    A2'(t) = k12 * A1(t) - k21 * A2(t),
    y(t) = A1(t) / V
)

println("═" ^ 55)
println("  Two-Compartment Pharmacokinetic Model")
println("═" ^ 55)
println()
println("Model: dA1/dt = -(k12+ke)*A1 + k21*A2 + u(t)")
println("       dA2/dt = k12*A1 - k21*A2")
println("       y = A1/V")
println("Input: u(t) [known drug dosing]")
println()

# Local identifiability (fast check)
println("─" ^ 50)
println("Local Identifiability (quick check):")
println("─" ^ 50)
local_pk = assess_local_identifiability(pk_2comp)
println(local_pk)
println()

# Global identifiability (full analysis)
println("─" ^ 50)
println("Global Identifiability:")
println("─" ^ 50)
global_pk = assess_identifiability(pk_2comp)
println(global_pk)
println()

# Identifiable functions
println("─" ^ 50)
println("Identifiable Functions:")
println("─" ^ 50)
id_funcs_pk = find_identifiable_functions(pk_2comp)
println(id_funcs_pk)

### Interpreting the Results

For the two-compartment model with a known input $u(t)$ and plasma concentration as the output:

**Classical result (well-known in pharmacokinetics):**
When the initial conditions are treated as unknown (as `StructuralIdentifiability.jl` does by default), the identifiability depends on the details. With the input $u(t)$ providing excitation, the micro-rate constants $k_{12}$, $k_{21}$, $k_e$, and $V$ are typically globally identifiable.

**Why the input matters:** The input $u(t)$ acts as a probe that excites the system. Without it (e.g., observing only free decay from unknown initial conditions), fewer parameters may be identifiable. This is analogous to how you need to "push" a system to observe its full response.

**Clinical significance:** The global identifiability of this model under standard experimental conditions is reassuring: it means that properly conducted pharmacokinetic studies can, in principle, determine all the micro-rate constants. However, practical identifiability (whether the data are rich enough in practice) is a separate question.

> **Key takeaway:** Known inputs improve identifiability. In experimental design, controlled inputs (drug doses, stimuli) provide information that helps distinguish parameters.

---
## 8. Example 5: Within-Host Viral Dynamics <a id="8-example-5"></a>

### Model Description

The standard model for viral infection within a host (used extensively for HIV, hepatitis B and C, and SARS-CoV-2):

$$\frac{dT}{dt} = \lambda - d_T \, T - \beta \, T \, V$$
$$\frac{dI}{dt} = \beta \, T \, V - \delta \, I$$
$$\frac{dV}{dt} = p \, I - c \, V$$

where:
- $T(t)$ = uninfected target cells
- $I(t)$ = infected cells
- $V(t)$ = free virus (viral load)
- $\lambda$ = target cell production rate
- $d_T$ = target cell natural death rate
- $\beta$ = infection rate constant
- $\delta$ = infected cell death rate
- $p$ = viral production rate per infected cell
- $c$ = viral clearance rate

**The clinical reality:** In most clinical settings, only the **viral load** $V(t)$ can be measured routinely (via PCR tests). Target cell counts require specialized assays.

### Scenario A: Observing Only Viral Load (Typical Clinical Setting)

In [ ]:
# ═══════════════════════════════════════════════════════════════
# Example 5A: Viral dynamics — observe V(t) only
# ═══════════════════════════════════════════════════════════════

viral_V = @ODEmodel(
    T'(t) = lambda - dT * T(t) - beta * T(t) * V(t),
    I'(t) = beta * T(t) * V(t) - delta * I(t),
    V'(t) = p * I(t) - c * V(t),
    y(t) = V(t)                     # Only viral load is measured
)

println("═" ^ 55)
println("  Viral Dynamics — Scenario A: V(t) only")
println("═" ^ 55)
println()

println("Global Identifiability:")
result_V = assess_identifiability(viral_V)
println(result_V)
println()

println("Identifiable Functions:")
id_funcs_V = find_identifiable_functions(viral_V)
println(id_funcs_V)

### Scenario A: Interpretation

With only viral load measured, the model is heavily nonidentifiable. Key findings:

- **$\beta$ and $p$ are individually nonidentifiable**, but their product $\beta \cdot p$ may be identifiable. This makes biological sense: from viral load alone, you can estimate the overall "burst size" (how many new infections result per virion) but not separately how efficiently the virus infects ($\beta$) versus how many virions an infected cell produces ($p$).

- **$c$ and $\delta$** may appear as identifiable combinations (e.g., $c + \delta$, $c \cdot \delta$).

This result has profound implications: it means that many published estimates of individual viral dynamics parameters from viral load data alone may be unreliable unless additional constraints were imposed.

### Scenario B: Observing Viral Load AND Target Cells

In [ ]:
# ═══════════════════════════════════════════════════════════════
# Example 5B: Viral dynamics — observe V(t) and T(t)
# ═══════════════════════════════════════════════════════════════

viral_VT = @ODEmodel(
    T'(t) = lambda - dT * T(t) - beta * T(t) * V(t),
    I'(t) = beta * T(t) * V(t) - delta * I(t),
    V'(t) = p * I(t) - c * V(t),
    y1(t) = V(t),                   # Viral load
    y2(t) = T(t)                    # Target cell count
)

println("═" ^ 55)
println("  Viral Dynamics — Scenario B: V(t) and T(t)")
println("═" ^ 55)
println()

println("Global Identifiability:")
result_VT = assess_identifiability(viral_VT)
println(result_VT)

### Scenario B: Interpretation

Adding target cell measurements dramatically improves identifiability. With both $V(t)$ and $T(t)$ observed, most (or all) parameters become identifiable. The additional output resolves the confounding between $\beta$ and $p$, because $T(t)$ provides direct information about the infection rate $\beta \cdot T \cdot V$.

### Practical Implications

| Observation | Parameters identifiable | Clinical cost |
|-------------|:----------------------:|:-------------:|
| $V(t)$ only | Only combinations | Low (routine blood test) |
| $V(t)$ and $T(t)$ | Most or all | Higher (specialized assay) |

This is a textbook case where identifiability analysis **guides experimental design**. If you need individual parameter estimates (e.g., to compare drug efficacy at the cellular level), the analysis tells you exactly which additional measurements are needed. If you only need certain parameter combinations (e.g., the overall viral fitness $\beta \cdot p$), viral load alone may suffice.

> **Key takeaway:** Identifiability analysis reveals the precise trade-off between experimental cost and information content. It tells you what measurements you *need*, not just what measurements you *want*.

---
## 9. Example 6: SEIR Model with Hospitalization <a id="9-example-6"></a>

### Model Description

During the COVID-19 pandemic, many modeling groups used SEIR-type models extended with hospitalization compartments. Here is a representative model:

$$\frac{dS}{dt} = -\frac{\beta \, S \, I}{N}$$
$$\frac{dE}{dt} = \frac{\beta \, S \, I}{N} - \sigma \, E$$
$$\frac{dI}{dt} = \sigma \, E - \gamma \, I - \alpha \, I$$
$$\frac{dH}{dt} = \alpha \, I - \eta \, H - \mu \, H$$
$$\frac{dR}{dt} = \gamma \, I + \eta \, H$$

**Parameters:**
| Symbol | Meaning | Typical range |
|--------|---------|:------------:|
| $\beta$ | Transmission rate | 0.1 – 1.0 /day |
| $\sigma$ | Incubation rate ($1/\sigma$ = latent period) | 1/5 – 1/2 /day |
| $\gamma$ | Recovery rate (non-hospitalized) | 1/14 – 1/5 /day |
| $\alpha$ | Hospitalization rate | 0.01 – 0.1 /day |
| $\eta$ | Hospital discharge rate | 1/14 – 1/5 /day |
| $\mu$ | Hospital mortality rate | 0.005 – 0.05 /day |
| $N$ | Total population (known) | Fixed |

**Available data (three streams):**
1. $y_1(t) = I(t)$ — reported infectious cases
2. $y_2(t) = H(t)$ — hospital census (number currently hospitalized)
3. $y_3(t) = \mu \cdot H(t)$ — hospital deaths

### Computational Analysis

In [ ]:
# ═══════════════════════════════════════════════════════════════
# Example 6: SEIR model with hospitalization
# ═══════════════════════════════════════════════════════════════

seir_h = @ODEmodel(
    S'(t) = -beta * S(t) * I(t) / N,
    E'(t) = beta * S(t) * I(t) / N - sigma * E(t),
    I'(t) = sigma * E(t) - gamma * I(t) - alpha * I(t),
    H'(t) = alpha * I(t) - eta * H(t) - mu * H(t),
    R'(t) = gamma * I(t) + eta * H(t),
    y1(t) = I(t),                   # Reported cases
    y2(t) = H(t),                   # Hospital census
    y3(t) = mu * H(t)               # Hospital deaths
)

println("═" ^ 55)
println("  SEIR-H Model: Cases, Hospitalizations, Deaths")
println("═" ^ 55)
println()

# Local check first (faster for complex models)
println("─" ^ 50)
println("Local Identifiability (quick screen):")
println("─" ^ 50)
local_seir = assess_local_identifiability(seir_h)
println(local_seir)
println()

# Full global analysis
println("─" ^ 50)
println("Global Identifiability:")
println("─" ^ 50)
global_seir = assess_identifiability(seir_h)
println(global_seir)

### Interpreting the Results

This model has a nice decomposition structure that aids identifiability analysis:

**Directly determined parameters:**
1. $\mu$ is immediately identifiable from $\mu = y_3(t) / y_2(t)$ (death rate = deaths / hospitalizations)
2. From $\dot{y}_2 = \alpha \, y_1 - (\eta + \mu) \, y_2$, the coefficients $\alpha$ and $\eta + \mu$ can be read off. Since $\mu$ is already known, $\eta$ is determined.
3. So $\alpha$, $\eta$, and $\mu$ are all globally identifiable.

**Remaining parameters ($\beta$, $\sigma$, $\gamma$):**
These require the full differential algebra analysis, but with three output streams, there is typically enough information to resolve them (at least locally).

### Lesson: Multiple Data Streams Help

Each data stream provides independent equations that constrain different parameter combinations:
- **Case data** ($I$) constrains $\sigma$, $\gamma$, $\alpha$, and the force of infection $\beta S I / N$
- **Hospital data** ($H$) constrains $\alpha$, $\eta$, $\mu$
- **Death data** ($\mu H$) directly identifies $\mu$

> **Key takeaway:** In complex models, multiple complementary data streams are essential. Plan data collection to cover different aspects of the model.

---
## 10. Example 7: Environmental Transmission Model <a id="10-example-7"></a>

### Model Description

Wastewater surveillance has emerged as a powerful tool for tracking infectious diseases (notably during the COVID-19 pandemic). This model incorporates both direct person-to-person transmission and environmental (e.g., waterborne) transmission:

$$\frac{dS}{dt} = -\beta_I \, S \, I - \beta_W \, S \, W$$
$$\frac{dI}{dt} = \beta_I \, S \, I + \beta_W \, S \, W - \gamma \, I$$
$$\frac{dR}{dt} = \gamma \, I$$
$$\frac{dW}{dt} = \xi \, I - \delta \, W$$

where:
- $\beta_I$ = direct (person-to-person) transmission rate
- $\beta_W$ = environmental transmission rate
- $\xi$ = pathogen shedding rate into the environment
- $\delta$ = environmental decay rate of the pathogen
- $W(t)$ = environmental pathogen concentration (e.g., viral RNA in wastewater)

### Does Wastewater Data Help? A Comparison

In [ ]:
# ═══════════════════════════════════════════════════════════════
# Example 7A: Environmental transmission — with wastewater data
# ═══════════════════════════════════════════════════════════════

sir_env = @ODEmodel(
    S'(t) = -betaI * S(t) * I(t) - betaW * S(t) * W(t),
    I'(t) = betaI * S(t) * I(t) + betaW * S(t) * W(t) - gamma * I(t),
    R'(t) = gamma * I(t),
    W'(t) = xi * I(t) - delta * W(t),
    y1(t) = I(t),                   # Case surveillance
    y2(t) = W(t)                    # Wastewater monitoring
)

println("═" ^ 55)
println("  Environmental Model — WITH Wastewater Data")
println("═" ^ 55)
println()

println("Global Identifiability:")
result_with_W = assess_identifiability(sir_env)
println(result_with_W)
println()

println("Identifiable Functions:")
id_funcs_env = find_identifiable_functions(sir_env)
println(id_funcs_env)

In [ ]:
# ═══════════════════════════════════════════════════════════════
# Example 7B: Environmental transmission — WITHOUT wastewater data
# ═══════════════════════════════════════════════════════════════

sir_env_noW = @ODEmodel(
    S'(t) = -betaI * S(t) * I(t) - betaW * S(t) * W(t),
    I'(t) = betaI * S(t) * I(t) + betaW * S(t) * W(t) - gamma * I(t),
    R'(t) = gamma * I(t),
    W'(t) = xi * I(t) - delta * W(t),
    y(t) = I(t)                     # Only case surveillance
)

println("═" ^ 55)
println("  Environmental Model — WITHOUT Wastewater Data")
println("═" ^ 55)
println()

println("Global Identifiability:")
result_no_W = assess_identifiability(sir_env_noW)
println(result_no_W)

### Interpreting the Comparison

**With wastewater data** ($y_1 = I$, $y_2 = W$):
- From $\dot{W} = \xi \, I - \delta \, W$ and observing both $W$ and $I$, we can directly extract $\xi$ and $\delta$
- The additional constraint from the $W$ equation helps resolve other parameters

**Without wastewater data** ($y = I$ only):
- The environmental compartment $W$ is completely hidden
- The two transmission pathways ($\beta_I \cdot S \cdot I$ and $\beta_W \cdot S \cdot W$) are confounded
- We expect significant nonidentifiability in the environmental parameters

This provides a rigorous, mathematical justification for investing in wastewater surveillance infrastructure. It is not merely a "nice-to-have" data source — it resolves fundamental parameter ambiguities that case data alone cannot address.

> **Key takeaway:** Wastewater monitoring is not redundant with case surveillance. It provides mathematically independent information that is essential for separating direct and environmental transmission pathways.

---
## 11. Interpreting and Resolving Nonidentifiability <a id="11-resolving"></a>

When your identifiability analysis reveals nonidentifiable parameters, **do not despair**. There are several principled strategies for moving forward.

### Strategy 1: Fix Parameters from External Sources

If a reliable estimate of a nonidentifiable parameter exists from independent experiments or published literature, you can fix it to that value.

**Example:** In the viral dynamics model, if the viral clearance rate $c$ is known from independent pharmacokinetic studies, fixing $c$ may make the remaining parameters identifiable.

**Requirements:**
- The external estimate must be credible and well-documented
- You should perform a sensitivity analysis (how do your conclusions change if the fixed value is wrong?)
- You must report that the parameter was fixed, not estimated

### Strategy 2: Reparameterize Using Identifiable Functions

Replace nonidentifiable individual parameters with identifiable combinations.

**Example:** If $\beta$ and $p$ are nonidentifiable but $\beta \cdot p$ is identifiable:
- Define $q = \beta \cdot p$ as a new parameter
- Rewrite the model in terms of $q$
- Estimate $q$ instead of trying to estimate $\beta$ and $p$ separately
- Interpret results in terms of $q$

This is often the most scientifically honest approach, because you are estimating exactly what the data can tell you.

### Strategy 3: Add Measurements

Design experiments to measure additional state variables. The identifiability analysis tells you exactly which measurements would help.

**Example:** Viral dynamics with $V$ observed only → add $T$ measurements to resolve nonidentifiability.

### Strategy 4: Bayesian Regularization

Use informative prior distributions in a Bayesian framework to constrain nonidentifiable parameters.

**Critical caveat:** For structurally nonidentifiable parameters, the posterior will be dominated by the prior, not the data. You are not "estimating" the parameter from data — you are essentially reporting your prior belief. This must be transparently communicated.

### Strategy 5: Simplify the Model

Reduce complexity by:
- Combining compartments that the data cannot distinguish
- Applying quasi-steady-state approximations to fast variables
- Eliminating parameters that the data cannot inform

**Example:** If $\beta$ and $p$ always appear as $\beta \cdot p$, replace the two-step infection process with a single step using rate $q = \beta \cdot p$.

### Decision Flowchart

```
Parameter nonidentifiable?
│
├── Is an identifiable function available?
│   ├── Yes → Reparameterize (Strategy 2)
│   └── No → Continue below
│
├── Is a reliable external estimate available?
│   ├── Yes → Fix parameter (Strategy 1)
│   └── No → Continue below
│
├── Can additional measurements be made?
│   ├── Yes → Augment observations (Strategy 3)
│   └── No → Continue below
│
├── Is Bayesian analysis appropriate?
│   ├── Yes → Use informative priors with caveats (Strategy 4)
│   └── No → Simplify the model (Strategy 5)
```

---
## 12. Summary and Best Practices <a id="12-summary"></a>

### The Complete Identifiability Workflow

Here is the workflow we recommend for every modeling project:

**Phase 1: Before collecting data**
1. Write down your model equations explicitly
2. Specify exactly what you can measure (observation functions)
3. Run structural identifiability analysis
4. If nonidentifiabilities are found, consider modifying the experimental design

**Phase 2: After collecting data**
5. Perform practical identifiability analysis (profile likelihood, Fisher information)
6. Estimate parameters only for those that pass both structural and practical checks
7. Report identifiable functions for nonidentifiable parameters

**Phase 3: Reporting**
8. State that identifiability analysis was performed
9. Report the classification of each parameter
10. Explain how nonidentifiable parameters were handled

### Quick Reference Card

| Task | Function | When to Use |
|------|----------|-------------|
| Fast screening | `assess_local_identifiability(model)` | First check; catches obvious problems |
| Full analysis | `assess_identifiability(model)` | Definitive classification (GI, LI, NI) |
| What can be estimated? | `find_identifiable_functions(model)` | When nonidentifiability is found |

### Common Pitfalls to Avoid

1. **Fitting a nonidentifiable model.** The optimizer will converge, but the estimates are meaningless. Always check identifiability first.

2. **Confusing structural and practical identifiability.** A structurally identifiable parameter can still be practically nonidentifiable with your specific data. Both checks are needed.

3. **Ignoring the observation function.** The same dynamics can be identifiable or not, depending on what you measure. Always specify observations explicitly.

4. **Treating fixed parameters as estimated.** If you fix a parameter from the literature, do not report it as "estimated" or include it in confidence intervals.

5. **Assuming more outputs always help.** Adding observations generally improves identifiability, but redundant observations (e.g., two measurements that are proportional) may not help.

### Summary of Examples

| Example | Model | Key Lesson |
|---------|-------|------------|
| 1 | Simple decay ($y = x$) | Direct observation → full identifiability |
| 2 | Scaled decay ($y = ax$) | Unknown scaling → product identifiable, individuals not |
| 3 | SIR epidemic | Identifiability depends critically on what is measured |
| 4 | Two-compartment PK | Known inputs (doses) improve identifiability |
| 5 | Viral dynamics | Guides experimental design: which measurements are needed? |
| 6 | SEIR-H | Multiple data streams resolve different parameters |
| 7 | Environmental transmission | Wastewater data resolves fundamental ambiguities |

---
## 13. References <a id="13-references"></a>

### Foundational Papers

1. **Bellman, R. and Åström, K.J.** (1970). "On structural identifiability." *Mathematical Biosciences*, 7(3-4), 329-339. — *The paper that started the field.*

2. **Ljung, L. and Glad, T.** (1994). "On global identifiability for arbitrary model parametrizations." *Automatica*, 30(2), 265-276. — *Introduced the differential algebra approach.*

3. **Cobelli, C. and DiStefano III, J.J.** (1980). "Parameter and structural identifiability concepts and ambiguities: A critical review and analysis." *American Journal of Physiology*, 239(1), R7-R24.

### Methods and Software

4. **Dong, R., Goodbrake, C., Harrington, H.A., and Pogudin, G.** (2023). "Differential elimination for dynamical models via projections with applications to structural identifiability." *SIAM Journal on Applied Algebra and Geometry*, 7(1), 194-235. — *The theory behind StructuralIdentifiability.jl.*

5. **Hong, H., Ovchinnikov, A., Pogudin, G., and Yap, C.** (2019). "SIAN: Software for structural identifiability analysis of ODE models." *Bioinformatics*, 35(16), 2873-2874.

6. **Villaverde, A.F., Barreiro, A., and Papachristodoulou, A.** (2016). "Structural identifiability of dynamic systems biology models." *PLoS Computational Biology*, 12(10), e1005153. — *STRIKE-GOLDD software.*

7. **Chis, O.T., Banga, J.R., and Balsa-Canto, E.** (2011). "Structural identifiability of systems biology models: A critical comparison of methods." *PLoS ONE*, 6(11), e27755.

### Practical Identifiability

8. **Raue, A. et al.** (2009). "Structural and practical identifiability analysis of partially observed dynamical models by exploiting the profile likelihood." *Bioinformatics*, 25(15), 1923-1929. — *Profile likelihood method for practical identifiability.*

9. **Wieland, F.G. et al.** (2021). "On structural and practical identifiability." *Current Opinion in Systems Biology*, 25, 60-69.

### Application-Specific

10. **Miao, H., Xia, X., Perelson, A.S., and Wu, H.** (2011). "On identifiability of nonlinear ODE models and applications in viral dynamics." *SIAM Review*, 53(1), 3-39.

11. **Chowell, G., Tariq, A., and Hyman, J.M.** (2023). "Structural identifiability analysis of epidemic models based on differential equations: A tutorial-based primer." *Journal of Mathematical Biology*, 87, Article 79.

### Books

12. **Godfrey, K.R.** (1983). *Compartmental Models and Their Application*. London: Academic Press.

13. **Walter, E. and Pronzato, L.** (1997). *Identification of Parametric Models from Experimental Data*. London: Springer.

---

**End of Notebook**

*This notebook accompanies the paper: "Structural and Practical Identifiability Analysis in Mathematical Modeling: A Comprehensive Survey and Tutorial" by A. Alsammani, Department of Mathematical Sciences, Delaware State University.*